In [1]:
VAMPIRE_PATH = '/home/apluska/.vampire/bin/vampire_z3_rel_static_casc2023_6749'
TPTP_PATH = '/home/apluska/TPTP-v8.2.0/'

We select problems which are at most 100_000 bytes in size, have at most 16 variables per clause, functions with at most arity 8, and at most 16 functions of each arity. 

In [21]:
from foreduce.tptp.parser import read_file
import os
from tqdm.auto import tqdm
from itertools import zip_longest


total, success = 0, 0
variables, functions = 0, []
problems = []
for dir in (pbar := tqdm(os.listdir(TPTP_PATH + 'Problems'))):
    for file in os.listdir(TPTP_PATH + 'Problems/' + dir):
        current = file
        pbar.set_description(f'Selected {success}/{total} Problems, currently parsing {dir}/{file}')
        if not file.endswith('.p'):
            continue
        try:
            total += 1
            problem = read_file(TPTP_PATH + 'Problems/' + dir + '/' + file, include_path=TPTP_PATH, max_size=100_000)
            success += 1
            _variables = max(len(clause.variables()) for clause in problem.clauses)
            if _variables > 16:
                continue
            variables = max(variables, _variables)
            _functions = []
            for f in problem.function_symbols() | problem.predicate_symbols():
                if f.arity > 8:
                    continue
                if len(_functions) <= f.arity:
                    _functions += [0 for _ in range(f.arity + 1 - len(_functions))]
                _functions[f.arity] += 1
            if any(a > 16 for a in _functions):
                continue
            functions = [max(a, b) for a, b in zip_longest(functions, _functions, fillvalue=0)]
            problems.append(dir + '/' + file)
        except Exception as e:
            continue

with open('problems.txt', 'w') as f:
    f.write('\n'.join(problems))

print(f'Maximum number of variables: {variables}')
print(f'Maximum number of functions of each arity: {functions}')

Selected 0/4 Problems, currently parsing RNG/RNG025-9.p:   0%|          | 0/55 [00:00<?, ?it/s]

Selected 8468/25473 Problems, currently parsing SEV/SEV514^1.p: 100%|██████████| 55/55 [06:30<00:00,  7.11s/it]         


Maximum number of variables: 16
Maximum number of functions of each arity: [16, 16, 16, 16, 9, 4, 4, 5, 5]


8468/25473 have been selected. The maximum number of functions of each respective arity is [16, 16, 16, 16, 9, 4, 4, 5, 5].

Next, we generate proofs for these problems using vampire with a timeout of 1. We limit ourselves to proofs which are at most 1_000_000 characters long.

In [14]:
import subprocess

with open('problems.txt', 'r') as f:
    problems = f.read().split('\n')

total, success = 0, 0
for problem in (pbar := tqdm(problems)):
    pbar.set_description(f'Succesfully proved {success}/{total} Problems, currently proving {problem}')
    args = [VAMPIRE_PATH, TPTP_PATH + 'Problems/' + problem,  '--show_new', 'on', '--include', TPTP_PATH, '-t', '1', '--avatar', 'off', '--proof', 'off']
    try:
        result = subprocess.run(args, capture_output=True, text=True, timeout=5)
    except subprocess.TimeoutExpired:
        continue
    if result.returncode == 0:
        if len(result.stdout) < 1_000_000:
            success += 1    
            os.makedirs(os.path.dirname('./proofs/' + problem), exist_ok=True)
            with open('./proofs/' + problem, 'w') as f:
                f.write(result.stdout)
    total += 1
pbar.set_description(f'Succesfully proved {success}/{total} Problems')

total, success

Succesfully proved 4500/8949 Problems, currently proving SEV/SEV436-1.p: 100%|██████████| 8954/8954 [1:31:18<00:00,  1.63it/s]      


(8950, 4501)

We manage to prove a bit over half the selected problems.

Let's bring our proofs into tensor form. We fix a maximum number of 1024 steps per proof and 128 tokens per clause. We will be generating 64 data points per problem, i.e. 8MB of data.

In [1]:
from foreduce.data.data import VampireProofs
from foreduce.transformer.tokenizer import TokenConfig
from foreduce.vampire.parser import read_file
import os

config = TokenConfig(num_variables=16, num_functions=[16, 16, 16, 16, 9, 4, 4, 5, 5])
dataset = VampireProofs(config=config, max_steps=1024, max_tokens=128)

for dir in os.listdir('./proofs')[:1]:
    for file in os.listdir('./proofs/' + dir)[:2]:
        problem, tree = read_file('./proofs/' + dir + '/' + file)
        for _ in range(64):
            dataset.add_proof(problem, tree)

dataset.save('proofs.pt')

KeyboardInterrupt: 

In [10]:
problem, tree = read_file('./proofs/GEO/GEO002-1.p')
dataset.add_proof(problem, tree)

In [5]:
inverted = {v: k for k, v in mapping.items()}

input = [inverted[i] if i in inverted else f"X{i}" for i in goal.tolist()]
result = ""
for i in range(input):
    result += input[i]
    if input[i+1] not in ["(", ")"]:
        result += " "

'<START>achievable(west(m(s(s(X12())))c(s(s(s(s(X12()))))))boatonwest()east(m(X20())c(X23())))<END>'

Now it's time to extract data from the proofs. Again, we only go for proofs with less than 1_000_000 bytes.

In [3]:
from foreduce.vampire.parser import parse_string
import os
from tqdm.auto import tqdm

success, total = 0, 0
attempts = []
for dir in (pbar := tqdm(os.listdir('./proofs/'))):
    for file in os.listdir('./proofs/' + dir):
        pbar.set_description(f'Parsed {success}/{total}, curently parsing {dir}/{file}')
        total += 1
        if os.path.getsize('./proofs/' + dir + '/' + file) > 1_000_000:
            continue
        with open('./proofs/' + dir + '/' + file, 'r') as f:
            problem = f.read()
        success += 1
        attempts.append(parse_string(problem))


Parsed 29/43, curently parsing RNG/RNG011-5.p: 100%|██████████| 1/1 [00:21<00:00, 21.48s/it]


In [31]:
from torchtune.modules import RotaryPositionalEmbeddings
import torch

rotary = RotaryPositionalEmbeddings(4, base=50)
x = torch.ones(2, 7, 4)
rotary(x.view(2, 7, 1, -1)).view(2, 7, -1)


tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000],
         [-0.3012,  1.3818,  0.8491,  1.1310],
         [-1.3254,  0.4932,  0.6812,  1.2394],
         [-1.1311, -0.8489,  0.4997,  1.3230],
         [ 0.1032, -1.4104,  0.3082,  1.3802],
         [ 1.2426, -0.6753,  0.1106,  1.4099],
         [ 1.2396,  0.6808, -0.0892,  1.4114]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000],
         [-0.3012,  1.3818,  0.8491,  1.1310],
         [-1.3254,  0.4932,  0.6812,  1.2394],
         [-1.1311, -0.8489,  0.4997,  1.3230],
         [ 0.1032, -1.4104,  0.3082,  1.3802],
         [ 1.2426, -0.6753,  0.1106,  1.4099],
         [ 1.2396,  0.6808, -0.0892,  1.4114]]])